In [45]:
#使用している気象庁の過去データを引っ張ってくるAPIは「気象台・測候所等」の地点のみしか対応していない点に留意

#	地点名 (no)	地点番号
#	稚内	47401
#	北見枝幸	47402
#	羽幌	47404
#	雄武	47405
#	留萌	47406
#	旭川	47407
#	網走	47409
#	小樽	47411
#	札幌	47412
#	岩見沢	47413
#	帯広	47417
#	釧路	47418
#	根室	47420
#	寿都	47421
#	室蘭	47423
#	苫小牧	47424
#	浦河	47426
#	江差	47428
#	函館	47430
#	倶知安	47433
#	紋別	47435
#	広尾	47440
#	大船渡	47512
#	新庄	47520
#	若松	47570
#	深浦	47574
#	青森	47575
#	むつ	47576
#	八戸	47581
#	秋田	47582
#	盛岡	47584
#	宮古	47585
#	酒田	47587
#	山形	47588
#	仙台	47590
#	石巻	47592
#	福島	47595
#	白河	47597
#	小名浜	47598
#	輪島	47600
#	相川	47602
#	新潟	47604
#	金沢	47605
#	伏木	47606
#	富山	47607
#	長野	47610
#	高田	47612
#	宇都宮	47615
#	福井	47616
#	高山	47617
#	松本	47618
#	諏訪	47620
#	軽井沢	47622
#	前橋	47624
#	熊谷	47626
#	水戸	47629
#	敦賀	47631
#	岐阜	47632
#	名古屋	47636
#	飯田	47637
#	甲府	47638
#	富士山	47639
#	河口湖	47640
#	秩父	47641
#	つくば（館野）	47646
#	銚子	47648
#	上野	47649
#	津	47651
#	伊良湖	47653
#	浜松	47654
#	御前崎	47655
#	静岡	47656
#	三島	47657
#	東京	47662
#	尾鷲	47663
#	石廊崎	47666
#	網代	47668
#	横浜	47670
#	館山	47672
#	勝浦	47674
#	大島	47675
#	三宅島	47677
#	八丈島	47678
#	千葉	47682
#	四日市	47684
#	奥日光（日光）	47690
#	西郷	47740
#	松江	47741
#	境	47742
#	米子	47744
#	鳥取	47746
#	豊岡	47747
#	舞鶴	47750
#	伊吹山	47751
#	萩	47754
#	浜田	47755
#	津山	47756
#	京都	47759
#	彦根	47761
#	下関	47762
#	広島	47765
#	呉	47766
#	福山	47767
#	岡山	47768
#	姫路	47769
#	神戸	47770
#	大阪	47772
#	洲本	47776
#	和歌山	47777
#	潮岬	47778
#	奈良	47780
#	山口	47784
#	厳原	47800
#	平戸	47805
#	福岡	47807
#	飯塚	47809
#	佐世保	47812
#	佐賀	47813
#	日田	47814
#	大分	47815
#	長崎	47817
#	雲仙岳	47818
#	熊本	47819
#	阿蘇山	47821
#	延岡	47822
#	阿久根	47823
#	人吉	47824
#	鹿児島	47827
#	都城	47829
#	宮崎	47830
#	枕崎	47831
#	油津	47835
#	屋久島	47836
#	種子島	47837
#	牛深	47838
#	福江	47843
#	松山	47887
#	多度津	47890
#	高松	47891
#	宇和島	47892
#	高知	47893
#	剣山	47894
#	徳島	47895
#	宿毛	47897
#	清水	47898
#	室戸岬	47899
#	名瀬	47909
#	与那国島	47912
#	石垣島	47918
#	宮古島	47927
#	久米島	47929
#	那覇	47936
#	名護	47940
#	沖永良部	47942
#	南大東（南大東島）	47945
#	父島	47971
#	南鳥島	47991

In [1]:
#使用ライブラリ等

import requests
import json
import pprint
import pandas as pd
import folium 
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import json
import sys
import datetime
import csv
from tkinter import filedialog
import numpy as np
import urllib3
from urllib3.exceptions import InsecureRequestWarning


#使用API
#衛星画像検索「Sentinelsat」 https://sentinelsat.readthedocs.io/en/stable/
#気象庁過去データ https://www.nakakamado.com/2022/09/weather-past-webapi.html

urllib3.disable_warnings(InsecureRequestWarning) #API参照時に出てくるWarningを非表示にする一文

In [2]:
#AOIのGeojsonを開く

filename = filedialog.askopenfilename(
    title = "Geojsonを開く",
    filetypes = [("GEOJSONファイル",".geojson")]
    )

with open(str(filename)) as f:
    j = json.load(f)

for feature in j["features"]:
    coordinates = feature["geometry"]["coordinates"]
    point = [p for i in coordinates for j in i for p in j]

x = np.array(point)
coords = np.mean(x, axis=0) #読み込んだAOIの凡その中心座標

footprint_geojson = geojson_to_wkt(read_geojson(str(filename))) #SentinelAPIで使用するWKT

m = folium.Map(location=[coords[1],coords[0]], zoom_start=10) #地図で表示
folium.GeoJson(str(filename)).add_to(m)
m

In [3]:
#衛星データ

user = '' 
password = '' #Copernicus Open Access Hubのユーザー名とパスワード

area = '47610' #気象データを参照するのに使用。In1をみて地点番号を入力してください。
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

#検索期間を入力　YYYY M D h m s
start = datetime.datetime(2017, 4, 1, 0, 0, 0)
end = datetime.datetime(2022, 4, 1, 0, 0, 0)

products = api.query(footprint_geojson, #ココ可変。検索条件に合わせて
                     platformname = 'Sentinel-1',
                     #orbitdirection = 'Ascending',
                     #producttype = 'SLC',
                     date = (start, end)
                     )

products_gdf = api.to_geodataframe(products)
products_gdf_sorted = products_gdf.sort_values(['title']) #取得リストをタイトルでソートしています。
products_gdf_sorted

Querying products:   4%|3         | 100/2510 [00:00<?, ?product/s]

,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,missiondatatakeid,...,platformname,platformidentifier,instrumentname,instrumentshortname,productclass,polarisationmode,acquisitiontype,uuid,productconsolidation,geometry
9220ce67-f47d-4118-bb7f-550a461da240,S1A_IW_GRDH_1SDV_20170412T205042_20170412T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-12T20:50:42.321Z, Instrument: SA...",false,2017-04-12 20:50:42.321,2017-04-12 20:51:11.340,2017-04-13 01:02:15.303,109011,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,9220ce67-f47d-4118-bb7f-550a461da240,NaN,"POLYGON ((139.30930 36.65808, 136.51891 37.053..."
de1f0f99-3785-4b98-8912-cd8b869ad91b,S1A_IW_GRDH_1SDV_20170412T205111_20170412T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-12T20:51:11.342Z, Instrument: SA...",false,2017-04-12 20:51:11.342,2017-04-12 20:51:36.340,2017-04-13 01:02:15.488,109011,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,de1f0f99-3785-4b98-8912-cd8b869ad91b,NaN,"POLYGON ((138.95596 35.15300, 136.21970 35.549..."
b6e2a237-f5e4-4f3f-9160-ac0d390c49cc,S1A_IW_GRDH_1SDV_20170424T205042_20170424T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-24T20:50:42.846Z, Instrument: SA...",false,2017-04-24 20:50:42.846,2017-04-24 20:51:11.865,2017-04-25 01:20:09.146,110387,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,b6e2a237-f5e4-4f3f-9160-ac0d390c49cc,NaN,"POLYGON ((139.30902 36.65814, 136.51863 37.053..."
e7fd92a6-a70f-450b-91c0-e09c32ee5e91,S1A_IW_GRDH_1SDV_20170424T205111_20170424T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-04-24T20:51:11.867Z, Instrument: SA...",false,2017-04-24 20:51:11.867,2017-04-24 20:51:36.865,2017-04-25 01:20:11.356,110387,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,e7fd92a6-a70f-450b-91c0-e09c32ee5e91,NaN,"POLYGON ((138.95575 35.15305, 136.21959 35.549..."
d5fde1fd-1719-4987-a3f6-5b09eb1ad856,S1A_IW_GRDH_1SDV_20170506T205043_20170506T2051...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2017-05-06T20:50:43.657Z, Instrument: SA...",false,2017-05-06 20:50:43.657,2017-05-06 20:51:12.681,2017-05-07 00:52:11.698,111737,...,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,d5fde1fd-1719-4987-a3f6-5b09eb1ad856,NaN,"POLYGON ((139.30919 36.65806, 136.51869 37.053..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d660452d-c580-4150-a787-38e801681f1f,S1B_IW_SLC__1SDV_20211130T084211_20211130T0842...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-11-30T08:42:11.624Z, Instrument: SA...",false,2021-11-30 08:42:11.624,2021-11-30 08:42:38.578,2021-11-30 11:39:54.756,233230,...,Sentinel-1,2016-025A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,d660452d-c580-4150-a787-38e801681f1f,NaN,"MULTIPOLYGON (((137.64059 36.75822, 140.44482 ..."
e3f8a4c3-ba6f-48e2-b2a2-e0a6d8c0d566,S1B_IW_SLC__1SDV_20211205T085008_20211205T0850...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-12-05T08:50:08.389Z, Instrument: SA...",false,2021-12-05 08:50:08.389,2021-12-05 08:5

In [4]:
#気象データ呼び出し関数

def api_call(area,dt):
    url = 'https://api.cultivationdata.net/past?no=' + str(area) + '&year=' + str(dt.year) + '&month=' + str(dt.month)
    r = requests.get(url, verify=False)
    data = json.loads(r.text)
    return data

In [5]:
#気象データ検索関数

def api_search(data,date,key):
    for val in data.values():
        result = str(val[date][key])
    return result

In [6]:
#CSV出力

with open(str('List') +'.csv', 'w',newline="") as f:
    writer = csv.writer(f)
    no = 0
    writer.writerow(('No','Title','軌道','日付','合計降水量','合計降雪量','日照時間','前日の天気概要（昼）',
                     '前日の天気概況（夜）','当日の天気概況（昼）','当日の天気概況（夜）','WKT'))
    
    for i in range(len(products)):
        title = products_gdf_sorted.iloc[i]["title"]
        
        if 'GRDH' in title: #ココ可変。今はtitleにGRDHの文字が入るもののみ採用している。
            geometry = products_gdf_sorted.iloc[i]["geometry"]
            orbitdirection = products_gdf_sorted.iloc[i]["orbitdirection"]
            beginposition = products_gdf_sorted.iloc[i]["beginposition"]
            
            dt = beginposition.tz_localize('GMT').tz_convert('Asia/Tokyo') #日本時間に変換
            day = datetime.timedelta(days = 1)
            dt_y = dt - day
            
            if no != 0:
                if str(dt.year) + str(dt.month) != str(year) + str(month):
                    data = api_call(area,dt)
                    
                if str(year_y) + str(month_y) != str(dt_y.year) + str(dt_y.month):
                    data_y = api_call(area,dt_y)
                    
            else:
                data = api_call(area,dt)
                data_y = api_call(area,dt_y)
            
            year = dt.year
            month = dt.month
            day = dt.day
            
            year_y = dt_y.year
            month_y = dt_y.month
            day_y = dt_y.day
            
            date = str(year) + '-' + str(month) + '-' + str(day)
            date_y = str(year_y) + '-' + str(month_y) + '-' + str(day_y)
            
            no = no + 1
            
            total_rainfall = api_search(data,date,'合計降水量')
            total_snowfall = api_search(data,date,'合計降雪')
            daylight_hours = api_search(data,date,'日照時間')
            weather_overview_day_y = api_search(data_y,date_y,'天気概況（昼）')
            weather_overview_night_y = api_search(data_y,date_y,'天気概況（夜）')
            weather_overview_day = api_search(data,date,'天気概況（昼）')
            weather_overview_night = api_search(data,date,'天気概況（夜）')            

            print(no, title, orbitdirection, date, total_rainfall, total_snowfall, daylight_hours, weather_overview_day_y,
                  weather_overview_night_y, weather_overview_day, weather_overview_night, geometry)
            
            writer.writerow((no, title, orbitdirection, date, total_rainfall, total_snowfall, daylight_hours, weather_overview_day_y,
                             weather_overview_night_y, weather_overview_day, weather_overview_night, geometry))        
        
f.close()

1 S1A_IW_GRDH_1SDV_20170412T205042_20170412T205111_016116_01A9D3_CA76 DESCENDING 2017-4-13 -- -- 10.0 曇一時雨 晴一時曇 晴 快晴 POLYGON ((139.309296 36.658085, 136.518906 37.053249, 136.895737 38.794342, 139.75296 38.400478, 139.309296 36.658085))
2 S1A_IW_GRDH_1SDV_20170412T205111_20170412T205136_016116_01A9D3_AF30 DESCENDING 2017-4-13 -- -- 10.0 曇一時雨 晴一時曇 晴 快晴 POLYGON ((138.955963 35.153004, 136.219696 35.54995, 136.519882 37.053032, 139.309265 36.657993, 138.955963 35.153004))
3 S1A_IW_GRDH_1SDV_20170424T205042_20170424T205111_016291_01AF33_959F DESCENDING 2017-4-25 0.0 -- 9.8 快晴 曇一時晴 薄曇一時晴 曇 POLYGON ((139.309021 36.658142, 136.518631 37.05328, 136.895432 38.794373, 139.75267 38.400536, 139.309021 36.658142))
4 S1A_IW_GRDH_1SDV_20170424T205111_20170424T205136_016291_01AF33_054E DESCENDING 2017-4-25 0.0 -- 9.8 快晴 曇一時晴 薄曇一時晴 曇 POLYGON ((138.95575 35.153049, 136.219589 35.549957, 136.519714 37.053047, 139.309006 36.658051, 138.95575 35.153049))
5 S1A_IW_GRDH_1SDV_20170506T205043_20170506T205112_0

41 S1A_IW_GRDH_1SDV_20171208T205050_20171208T205119_019616_02152A_FB04 DESCENDING 2017-12-9 0.0 -- 7.8 曇後雪 雪後晴時々曇 晴時々曇 晴 POLYGON ((139.311478 36.657879, 136.52066 37.053158, 136.897522 38.79388, 139.755173 38.399895, 139.311478 36.657879))
42 S1A_IW_GRDH_1SDV_20171208T205119_20171208T205144_019616_02152A_3F2C DESCENDING 2017-12-9 0.0 -- 7.8 曇後雪 雪後晴時々曇 晴時々曇 晴 POLYGON ((138.957642 35.15287, 136.2211 35.549904, 136.521759 37.052925, 139.311462 36.657787, 138.957642 35.15287))
43 S1A_IW_GRDH_1SDV_20171220T205049_20171220T205118_019791_021A9D_87A8 DESCENDING 2017-12-21 -- -- 8.6 快晴 快晴 晴 快晴 POLYGON ((139.310257 36.657887, 136.51944 37.053112, 136.896469 38.795006, 139.754181 38.401081, 139.310257 36.657887))
44 S1A_IW_GRDH_1SDV_20171220T205118_20171220T205143_019791_021A9D_E80F DESCENDING 2017-12-21 -- -- 8.6 快晴 快晴 晴 快晴 POLYGON ((138.95639 35.152889, 136.219727 35.549877, 136.520416 37.052891, 139.310242 36.657799, 138.95639 35.152889))
45 S1A_IW_GRDH_1SDV_20180101T205049_20180101T205118_019

76 S1A_IW_GRDH_1SDV_20180712T205053_20180712T205122_022766_0277C2_F3C7 DESCENDING 2018-7-13 -- -- 8.3 曇 曇 薄曇 薄曇後一時晴 POLYGON ((139.309692 36.657997, 136.519196 37.053146, 136.895966 38.794151, 139.753311 38.400307, 139.309692 36.657997))
77 S1A_IW_GRDH_1SDV_20180712T205122_20180712T205147_022766_0277C2_ACED DESCENDING 2018-7-13 -- -- 8.3 曇 曇 薄曇 薄曇後一時晴 POLYGON ((138.956406 35.152908, 136.220032 35.549839, 136.520172 37.052925, 139.309677 36.657909, 138.956406 35.152908))
78 S1A_IW_GRDH_1SDV_20180724T205054_20180724T205123_022941_027D45_4DDF DESCENDING 2018-7-25 -- -- 10.8 晴一時曇 晴一時薄曇 薄曇 薄曇時々晴 POLYGON ((139.309647 36.658051, 136.519257 37.053211, 136.896118 38.794483, 139.753357 38.400623, 139.309647 36.658051))
79 S1A_IW_GRDH_1SDV_20180724T205123_20180724T205148_022941_027D45_3FE0 DESCENDING 2018-7-25 -- -- 10.8 晴一時曇 晴一時薄曇 薄曇 薄曇時々晴 POLYGON ((138.956345 35.152866, 136.220184 35.549797, 136.52034 37.052975, 139.309616 36.657959, 138.956345 35.152866))
80 S1A_IW_GRDH_1SDV_20180805T205055_201

111 S1A_IW_GRDH_1SDV_20190201T205054_20190201T205123_025741_02DC9C_19F7 DESCENDING 2019-2-2 -- -- 9.0 雪後曇一時晴 晴 晴一時曇 晴一時曇 POLYGON ((139.30983 36.657978, 136.519104 37.053185, 136.895767 38.793282, 139.753296 38.399372, 139.30983 36.657978))
112 S1A_IW_GRDH_1SDV_20190201T205123_20190201T205148_025741_02DC9C_BBE5 DESCENDING 2019-2-2 -- -- 9.0 雪後曇一時晴 晴 晴一時曇 晴一時曇 POLYGON ((138.956497 35.152893, 136.21991 35.549885, 136.520081 37.052967, 139.309799 36.657887, 138.956497 35.152893))
113 S1A_IW_GRDH_1SDV_20190213T205054_20190213T205123_025916_02E2CF_4BC7 DESCENDING 2019-2-14 0.0 2 0.2 雪時々晴後曇 晴時々雪 雪後時々曇 曇後一時晴 POLYGON ((139.311371 36.657936, 136.52066 37.053211, 136.89743 38.793484, 139.754959 38.399506, 139.311371 36.657936))
114 S1A_IW_GRDH_1SDV_20190213T205123_20190213T205148_025916_02E2CF_A01C DESCENDING 2019-2-14 0.0 2 0.2 雪時々晴後曇 晴時々雪 雪後時々曇 曇後一時晴 POLYGON ((138.95755 35.152927, 136.221115 35.549953, 136.521759 37.052979, 139.311356 36.657848, 138.95755 35.152927))
115 S1A_IW_GRDH_1SDV_201902

147 S1A_IW_GRDH_1SDV_20190905T205102_20190905T205132_028891_034661_120D DESCENDING 2019-9-6 0.0 -- 10.5 曇時々晴一時雨 曇一時雨後晴 晴 晴一時雨 MULTIPOLYGON (((139.310059 36.657902, 139.753876 38.400665, 136.896637 38.794582, 136.519684 37.05312, 139.310059 36.657902)))
148 S1A_IW_GRDH_1SDV_20190905T205132_20190905T205157_028891_034661_4382 DESCENDING 2019-9-6 0.0 -- 10.5 曇時々晴一時雨 曇一時雨後晴 晴 晴一時雨 MULTIPOLYGON (((138.956192 35.152897, 139.310043 36.657814, 136.52066 37.052898, 136.219971 35.549873, 138.956192 35.152897)))
149 S1A_IW_GRDH_1SDV_20190917T205103_20190917T205132_029066_034C71_E4ED DESCENDING 2019-9-18 0.0 -- 0.0 曇時々晴 曇一時雨 曇時々雨 曇 MULTIPOLYGON (((139.310623 36.65789, 139.754532 38.401104, 136.897171 38.795013, 136.520142 37.053101, 139.310623 36.65789)))
150 S1A_IW_GRDH_1SDV_20190917T205132_20190917T205157_029066_034C71_A815 DESCENDING 2019-9-18 0.0 -- 0.0 曇時々晴 曇一時雨 曇時々雨 曇 MULTIPOLYGON (((138.956802 35.152878, 139.310608 36.657803, 136.521118 37.052879, 136.220474 35.549854, 138.956802 35.152878))

...

428 S1B_IW_GRDH_1SDV_20190708T084132_20190708T084157_017040_0200E9_BC84 ASCENDING 2019-7-8 0.0 -- 5.9 曇一時雨 曇一時雨後晴 曇時々晴 曇時々雨 MULTIPOLYGON (((138.001556 35.308346, 140.767395 35.708431, 140.457352 37.210106, 137.637665 36.811718, 138.001556 35.308346)))
429 S1B_IW_GRDH_1SDV_20190720T084132_20190720T084157_017215_020602_16F3 ASCENDING 2019-7-20 0.5 -- 0.0 曇後時々雨 曇時々雨 曇時々雨 曇時々雨 MULTIPOLYGON (((138.002991 35.308319, 140.768951 35.708397, 140.458939 37.210075, 137.639145 36.811695, 138.002991 35.308319)))
430 S1B_IW_GRDH_1SDV_20190801T084133_20190801T084158_017390_020B33_8B5E ASCENDING 2019-8-1 0.0 -- 10.1 晴一時雨後曇 曇後一時雨 晴時々曇 晴時々曇 MULTIPOLYGON (((138.004196 35.308216, 140.770264 35.708275, 140.460281 37.209953, 137.640381 36.811592, 138.004196 35.308216)))
431 S1B_IW_GRDH_1SDV_20190813T084134_20190813T084159_017565_021096_A318 ASCENDING 2019-8-13 -- -- 9.4 晴時々曇 晴時々曇 晴後一時曇 晴 MULTIPOLYGON (((138.002762 35.308357, 140.768616 35.708424, 140.458588 37.210098, 137.638901 36.811729, 138.002762 35.3083

461 S1B_IW_GRDH_1SDV_20200202T084955_20200202T085020_020088_026052_A623 ASCENDING 2020-2-2 -- -- 9.2 雪時々みぞれ一時晴 曇時々晴 晴一時曇 曇時々晴 MULTIPOLYGON (((135.809601 35.838165, 138.568573 36.234489, 138.254608 37.735672, 135.44075 37.340843, 135.809601 35.838165)))
462 S1B_IW_GRDH_1SDV_20200209T084133_20200209T084158_020190_02638E_B028 ASCENDING 2020-2-9 3.5 9 1.5 曇時々雪、みぞれを伴う 雪時々曇 雪時々曇、霧を伴う 晴時々曇 MULTIPOLYGON (((138.001923 35.308029, 140.768311 35.708149, 140.458252 37.20982, 137.638031 36.811394, 138.001923 35.308029)))
463 S1B_IW_GRDH_1SDV_20200214T084955_20200214T085020_020263_0265F5_4CCB ASCENDING 2020-2-14 -- -- 2.4 晴一時雨 晴後時々曇 曇後一時晴 曇時々晴 MULTIPOLYGON (((135.809052 35.838089, 138.568024 36.234428, 138.254044 37.735611, 135.440216 37.340767, 135.809052 35.838089)))
464 S1B_IW_GRDH_1SDV_20200221T084132_20200221T084157_020365_026928_CB14 ASCENDING 2020-2-21 -- -- 7.9 曇時々晴 曇一時みぞれ後晴 晴一時霧 曇時々晴 MULTIPOLYGON (((138.001953 35.30801, 140.768341 35.70816, 140.458267 37.209827, 137.638031 36.811371, 138.001

499 S1B_IW_GRDH_1SDV_20201006T084142_20201006T084207_023690_02D038_2CDF ASCENDING 2020-10-6 -- -- 3.0 曇 晴時々曇 曇後時々晴 曇 MULTIPOLYGON (((138.002029 35.308254, 140.768188 35.708393, 140.458099 37.21006, 137.638092 36.811615, 138.002029 35.308254)))
500 S1B_IW_GRDH_1SDV_20201011T084946_20201011T085011_023763_02D284_2702 ASCENDING 2020-10-11 0.0 -- 3.4 曇時々雨 晴一時曇 曇時々晴一時雨 曇時々晴 MULTIPOLYGON (((136.059784 34.705452, 138.800995 35.1064, 138.495743 36.608929, 135.702347 36.209835, 136.059784 34.705452)))
501 S1B_IW_GRDH_1SDV_20201011T085011_20201011T085036_023763_02D284_5113 ASCENDING 2020-10-11 0.0 -- 3.4 曇時々雨 晴一時曇 曇時々晴一時雨 曇時々晴 MULTIPOLYGON (((135.702316 36.209923, 138.496704 36.609146, 138.185791 38.110695, 135.335083 37.713009, 135.702316 36.209923)))
502 S1B_IW_GRDH_1SDV_20201011T205032_20201011T205057_023770_02D2BA_7E8C DESCENDING 2020-10-12 0.0 -- 4.3 曇時々晴一時雨 曇時々晴 曇後時々晴一時雨 曇時々雨 MULTIPOLYGON (((139.311935 36.646572, 139.695831 38.147266, 136.83844 38.542805, 136.512024 37.04319, 139.311935 36.

532 S1B_IW_GRDH_1SDV_20210404T084139_20210404T084204_026315_03240C_1EA4 ASCENDING 2021-4-4 0.0 -- 0.1 晴後一時曇 曇時々晴 曇時々雨 雨時々曇 MULTIPOLYGON (((138.002258 35.307945, 140.768402 35.708103, 140.458344 37.209774, 137.638351 36.81131, 138.002258 35.307945)))
533 S1B_IW_GRDH_1SDV_20210409T085001_20210409T085026_026388_03266A_E8E5 ASCENDING 2021-4-9 0.0 -- 7.5 晴後時々曇 曇一時雨後時々晴、みぞれを伴う 晴後一時みぞれ 晴時々曇、みぞれを伴う MULTIPOLYGON (((135.810455 35.838211, 138.569107 36.234455, 138.255188 37.735733, 135.441666 37.340988, 135.810455 35.838211)))
534 S1B_IW_GRDH_1SDV_20210416T084139_20210416T084204_026490_03299D_95DC ASCENDING 2021-4-16 -- -- 7.3 晴後一時曇 曇後一時晴 曇時々晴 曇一時晴後時々雨 MULTIPOLYGON (((138.002899 35.308125, 140.768829 35.708237, 140.458786 37.209908, 137.639008 36.811493, 138.002899 35.308125)))
535 S1B_IW_GRDH_1SDV_20210421T085002_20210421T085027_026563_032C0F_DA9C ASCENDING 2021-4-21 -- -- 12.2 晴 晴一時曇 晴 晴 MULTIPOLYGON (((135.810272 35.838161, 138.568817 36.234417, 138.254868 37.735695, 135.441467 37.340935, 135.

568 S1B_IW_GRDH_1SDV_20211106T084148_20211106T084213_029465_038427_5A9C ASCENDING 2021-11-6 -- -- 8.0 晴一時雨 晴 晴時々曇 曇時々晴 MULTIPOLYGON (((138.002274 35.308189, 140.768555 35.708324, 140.458481 37.209991, 137.638367 36.811554, 138.002274 35.308189)))
569 S1B_IW_GRDH_1SDV_20211111T085010_20211111T085035_029538_038679_43C0 ASCENDING 2021-11-11 -- -- 6.9 晴時々雨一時曇 晴後一時曇 晴時々曇 晴 MULTIPOLYGON (((135.808975 35.838276, 138.567963 36.234627, 138.253937 37.735893, 135.440094 37.341038, 135.808975 35.838276)))
570 S1B_IW_GRDH_1SDV_20211118T084148_20211118T084213_029640_038989_DA7C ASCENDING 2021-11-18 -- -- 7.5 晴一時曇 曇時々晴 曇後晴 晴 MULTIPOLYGON (((138.00209 35.308151, 140.768478 35.708286, 140.45842 37.209953, 137.638184 36.811512, 138.00209 35.308151)))
571 S1B_IW_GRDH_1SDV_20211123T085010_20211123T085035_029713_038BEA_06A4 ASCENDING 2021-11-23 -- -- 8.4 曇後雨 曇時々晴一時雨 晴時々曇 曇一時晴後時々雨 MULTIPOLYGON (((135.809326 35.838295, 138.56842 36.234604, 138.254471 37.735783, 135.440506 37.340973, 135.809326 35.838295)))
5

In [17]:
#確認用

pprint.pprint(data_y)

{'長野': {'2018-6-1': {'合計降水量': '0.0',
                     '合計降雪': '--',
                     '天気概況（夜）': '快晴',
                     '天気概況（昼）': '晴一時曇',
                     '平均気温': '15.7',
                     '平均湿度': '76',
                     '平均風速': '2.3',
                     '日照時間': '5.3',
                     '最低気温': '12.2',
                     '最大10分間降水量': '0.0',
                     '最大1時間降水量': '0.0',
                     '最大瞬間風速': '9.2',
                     '最大瞬間風速（風向）': '東北東',
                     '最大風速': '5.8',
                     '最大風速（風向）': '東',
                     '最小湿度': '51',
                     '最深積雪': '--',
                     '最高気温': '21.8',
                     '海面気圧（平均）': '1012.6',
                     '現地気圧（平均）': '964.0'},
        '2018-6-10': {'合計降水量': '0.0',
                      '合計降雪': '--',
                      '天気概況（夜）': '曇時々雨',
                      '天気概況（昼）': '曇',
                      '平均気温': '20.0',
                      '平均湿度': '71',
              